# Ciencia de Datos acelerada con GPUs: Puntualidad en tráfico aéreo durante COVID

Cuando la pandemia inició, muchas industrias se vieron obligadas a disminuir y hasta cerrar operaciones. Un ejemplo, es la insdustria aérea, que sufrió una disminución significativa en los vuelos. Esto habrá tenido algún impacto en su performance de puntualidad? los vuelos son más puntuales?

Fuentes:
1. [Open Flights Data](https://openflights.org/data.html)
2. [Bureau of Transportation Statistics](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time). Vamos a usar data del periodo de 1/01/2020 a 31/05/2020.

jupyter labextension install @jupyter-widgets/jupyterlab-manager

## Imports
Primero lo primero, vamos a importar las herramientas que necesitaremos.

In [ ]:
import cudf
import blazingsql as bsql
import s3fs
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

### `BlazingContext`
Es necesario que establezcas a BlazingContext para conectarte a la instancia BlazingSQL, y así crear tablas, ejecutar consultas y básicamente, hacer cualquier cosa con BlazingSQL.

In [ ]:
bc = bsql.BlazingContext()

## Ingesta y ejecucion de datos

Hay dos maneras para cargar y ejecutar data usando las herramientas del ecosistema de RAPIDS: cargar directamente a la memoria usando `cudf` o `.create_table()` usando `BlazingContext`.

### Data de vuelos
Rutas a archivos en AWS s3

In [ ]:
flight_data_path = 's3://bsql/data/air_transport/flight_ontime_2020-0[1-5].parquet'
s3 = s3fs.S3FileSystem(anon=True)
files = [f's3://{f}' for f in s3.glob(flight_data_path)]
files

#### Leyendo la data
Usaremos cuDF.

In [ ]:
%%time
flights = []

for f in files:
    flights.append(cudf.read_parquet(f, storage_options={'anon': True}))
    
flights = cudf.concat(flights)

In [ ]:
flights.head(5)

In [ ]:
print(f'Número total de vuelos en el dataset: {len(flights):,}')

#### Registrando una tabla con `BlazingContext`

In [ ]:
bc.create_table('flights', flights)

#### Columnas y tipo de data

In [ ]:
flights.columns

In [ ]:
flights.dtypes

### Data de vuelos y aeropuertos
Definimos la ruta de los archivos en s3

In [ ]:
airports_path = 's3://bsql/data/air_transport/airports.csv'
airlines_path = 's3://bsql/data/air_transport/airlines.csv'

#### Leyendo la data

In [ ]:
airports_dtypes = OrderedDict([
      ('Airport ID', 'int64')
    , ('Name', 'str')
    , ('City', 'str')
    , ('Country', 'str')
    , ('IATA', 'str')
    , ('ICAO', 'str')
    , ('Latitude', 'float64')
    , ('Longitude', 'float64')
    , ('Altitude', 'int64')
    , ('Timezone', 'str')
    , ('DST', 'str')
    , ('Type', 'str')
    , ('Source', 'str')
])

airports = cudf.read_csv(
    airports_path
    , names=list(airports_dtypes.keys())
    , dtype=list(airports_dtypes.values())
    , storage_options={'anon': True}
)
airports.head()

In [ ]:
airlines_dtypes = OrderedDict([
    ('Airline ID', 'int64')
    , ('Name', 'str')
    , ('Alias', 'str')
    , ('IATA', 'str')
    , ('ICAO', 'str')
    , ('Callsign', 'str')
    , ('Country', 'str')
    , ('Active', 'str')
])

airlines = cudf.read_csv(
    airlines_path
    , names=list(airlines_dtypes.keys())
    , dtype=list(airlines_dtypes.values())
    , storage_options={'anon': True}
)
airlines.head()

Registramos las tablas dentro de `BlazingContext`

In [ ]:
bc.create_table('airports', airports)
bc.create_table('airlines', airlines)

Añadir el estado donde se encuentra el aeropuerto en la tabla `airports`.

In [ ]:
airports_geo = bc.sql('''
    SELECT DISTINCT A.airport
        , A.State
        , B.Name
        , B.City
    FROM (
        SELECT DISTINCT ORIGIN AS Airport
            , ORIGIN_STATE_ABR AS State
        FROM flights

        UNION ALL
        SELECT DISTINCT DEST AS Airport
            , DEST_STATE_ABR AS State
        FROM flights AS A
    ) AS A
    LEFT OUTER JOIN airports AS B
        ON A.Airport = B.IATA
''')
bc.create_table('airports_geo', airports_geo)

In [ ]:
%%time
bc.sql('''
    SELECT A.FL_DATE
        , A.OP_UNIQUE_CARRIER
        , B.Name AS CARRIER_NAME
        , A.ORIGIN
        , C.Name AS ORIGIN_NAME
        , C.City AS ORIGIN_CITY
        , C.State AS ORIGIN_STATE        
        , A.DEST
        , D.Name AS DEST_NAME
        , D.City AS DEST_CITY
        , A.DEP_DELAY AS Delay
    FROM flights AS A
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    LEFT OUTER JOIN airports_geo AS C
        ON A.ORIGIN = C.airport
    LEFT OUTER JOIN airports_geo AS D
        ON A.DEST = D.airport
    LIMIT 4
''')

## Preguntas

### 1. Cuántos aeropuertos hay en el dataset?

In [ ]:
print(f'Hay {len(flights["ORIGIN"].unique())} aeropuertos en el dataset')

### 2. Cuántos vuelos tuvieron retraso y cuántos partieron a tiempo? Cuál es la distribución?

In [ ]:
print(f'{len(flights[flights["DEP_DELAY"] > 0]):,} vuelos con retraso y {len(flights[flights["DEP_DELAY"] <= 0]):,} vuelos a tiempo')

In [ ]:
### calculando la distribución
n_bins = 100

delays = flights[flights['DEP_DELAY'] >  0]['DEP_DELAY']
ontime = flights[flights['DEP_DELAY'] <= 0]['DEP_DELAY']

In [ ]:
%%time
del_bins = np.array([i * 15 for i in range(0, n_bins)], dtype='float64')
delays_binned = delays.digitize(del_bins)
delays_histogram = delays_binned.groupby().count() / len(delays)
(
    delays_histogram
    .set_index(del_bins[delays_histogram.index.to_array()-1])
    .to_pandas()
    .plot(kind='bar', figsize=(20,9), ylim=[0,1.0], title='Distribución de salidas con demora')
)

In [ ]:
%%time
ontime_bins = np.array([i * (-1) for i in range(n_bins,0,-1)], dtype='float64')
ontime_binned = ontime.digitize(ontime_bins)
ontime_histogram = ontime_binned.groupby().count() / len(ontime)
(
    ontime_histogram
    .set_index(ontime_bins[ontime_histogram.index.to_array()-1])
    .to_pandas()
    .plot(kind='bar', figsize=(20,9), ylim=[0,1.0], title='Distribución de salidas a tiempo')
)

### 3. Distribución de demora a través del tiempo

In [ ]:
q10 = lambda x: x.quantile(0.10); q10.__name__ = "q0.10"
q25 = lambda x: x.quantile(0.25); q25.__name__ = "q0.25"
q75 = lambda x: x.quantile(0.75); q75.__name__ = "q0.75"
q90 = lambda x: x.quantile(0.90); q90.__name__ = "q0.90"

delay_aggs = (
    flights[['MONTH', 'DEP_DELAY']]
    .groupby(by='MONTH')
    .agg({'DEP_DELAY': [q10, q25,'median',q75, q90]})
)

delay_aggs.columns = ['_'.join(col) for col in delay_aggs.columns]

In [ ]:
host=delay_aggs.to_pandas()
host

In [ ]:
x = ['JAN', 'FEB', 'MAR', 'APR', 'MAY']
fig, ax = plt.subplots(figsize=(12,9))
ax.plot(x, host['DEP_DELAY_median'], 'r--', label='median')
ax.fill_between(x, host['DEP_DELAY_q0.10'], host['DEP_DELAY_q0.90']
                , alpha=.2, label='Quart10 - Quart90')
ax.fill_between(x, host['DEP_DELAY_q0.25'], host['DEP_DELAY_q0.75']
                , alpha=.2, label='Quart25 - Quart75')
ax.legend()

### 4. Cuáles son las top 5 aerolíneas y aeropuertos con mayores retrasos y al menos 1000 vuelos? Cuál es el promedio de demora?

In [ ]:
delays = flights[flights['DEP_DELAY'] >  0][['DEP_DELAY', 'ORIGIN', 'DEST', 'OP_UNIQUE_CARRIER']]
ontime = flights[flights['DEP_DELAY'] <= 0][['DEP_DELAY', 'ORIGIN', 'DEST', 'OP_UNIQUE_CARRIER']]

In [ ]:
bc.create_table('delays', delays)
bc.create_table('ontime', ontime)

In [ ]:
def plot_airports(df, cmap, agg_cols, indicator):
    host = df.to_pandas()
    
    ### figure
    fig = plt.figure(figsize=(12,9))
    
    ### plot on map
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_extent([-125, -70, 22, 55], crs=ccrs.PlateCarree()) ## US Continental
    ax.stock_img()
    
    ### add features
    ax.add_feature(cfeature.LAKES)
    ax.add_feature(cfeature.STATES)
    ax.add_feature(cfeature.COASTLINE)
    
    ### add data
    bubbles = ax.scatter(
        host['Longitude']
        , host['Latitude']
        , s=host[agg_cols['CNT']] / 50, label="Flight Count"
        , c=host[agg_cols['AVG']]
        , cmap=plt.get_cmap(cmap)
        , alpha=0.8
    )

    ### add colorbar
    delays = host[agg_cols['AVG']]
    tick_values = np.linspace(delays.min(), delays.max(), 11)
    cbar = fig.colorbar(bubbles, shrink=0.7)
    cbar.ax.set_yticklabels([f'{round(v)} mins' for v in tick_values], fontsize=14)
    cbar.set_label(f'Average {indicator}', fontsize=16)
    plt.legend(fontsize=16)

### Mayores retrasos
#### Origen

In [ ]:
%%time
delayed_origin = bc.sql('''
    SELECT A.ORIGIN
        , B.Name AS ORIGIN_Airport
        , B.City AS ORIGIN_City
        , B.Country AS ORIGIN_Country
        , B.Latitude
        , B.Longitude
        , COUNT(*) AS DELAY_CNT
        , AVG(DEP_DELAY) AS AVG_DELAY
    FROM delays AS A
    LEFT OUTER JOIN airports AS B
        ON A.ORIGIN = B.IATA
    GROUP BY A.ORIGIN
        , B.Name
        , B.City
        , B.Country
        , B.Latitude
        , B.Longitude
    HAVING COUNT(*) > 1000
        AND B.Country = 'United States'
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 60
''')
delayed_origin.head()

In [ ]:
### plot
plot_airports(
    delayed_origin
    , 'YlOrRd'
    , {'CNT': 'DELAY_CNT', 'AVG': 'AVG_DELAY'}
    , 'delays'
)

#### Destino

In [ ]:
%%time
delayed_destination = bc.sql('''
    SELECT A.DEST
        , B.Name AS DEST_Airport
        , B.City AS DEST_City
        , B.Country AS DEST_Country
        , B.Latitude
        , B.Longitude
        , COUNT(*) AS DELAY_CNT
        , AVG(DEP_DELAY) AS AVG_DELAY
    FROM delays AS A
    LEFT OUTER JOIN airports AS B
        ON A.DEST = B.IATA
    GROUP BY A.DEST
        , B.Name
        , B.City
        , B.Country
        , B.Latitude
        , B.Longitude
    HAVING COUNT(*) > 1000
        AND B.Country = 'United States'
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 60
''')
delayed_destination.head()

In [ ]:
### plot
plot_airports(
    delayed_destination
    , 'YlOrRd'
    , {'CNT': 'DELAY_CNT', 'AVG': 'AVG_DELAY'}
    , 'delays'
)

#### Aerolíneas

In [ ]:
%%time
bc.sql('''
    SELECT A.OP_UNIQUE_CARRIER AS CARRIER
        , B.Name AS CARRIER_Name
        , B.Country AS CARRIER_Country
        , COUNT(*) AS DELAY_CNT
        , AVG(DEP_DELAY) AS AVG_DELAY
    FROM delays AS A
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    GROUP BY A.OP_UNIQUE_CARRIER
        , B.Name
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 5
''')

### Los más puntuales

#### Origen

In [ ]:
%%time
ontime_origin = bc.sql('''
    SELECT A.ORIGIN
        , B.Name AS ORIGIN_Airport
        , B.City AS ORIGIN_City
        , B.Country AS ORIGIN_Country
        , B.Latitude
        , B.Longitude
        , COUNT(*) AS ONTIME_CNT
        , AVG(DEP_DELAY) AS AVG_ONTIME
    FROM ontime AS A
    LEFT OUTER JOIN airports AS B
        ON A.ORIGIN = B.IATA
    GROUP BY A.ORIGIN
        , B.Name
        , B.City
        , B.Country
        , B.Latitude
        , B.Longitude
    HAVING COUNT(*) > 1000
        AND B.Country = 'United States'
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 60
''')
ontime_origin.head()

In [ ]:
### plot
plot_airports(
    ontime_origin
    , 'Greens'
    , {'CNT': 'ONTIME_CNT', 'AVG': 'AVG_ONTIME'}
    , 'ontime'
)

#### Destino

In [ ]:
%%time
ontime_destination = bc.sql('''
    SELECT A.DEST
        , B.Name AS DEST_Airport
        , B.City AS DEST_City
        , B.Country AS DEST_Country
        , B.Latitude
        , B.Longitude
        , COUNT(*) AS ONTIME_CNT
        , AVG(DEP_DELAY) AS AVG_ONTIME
    FROM ontime AS A
    LEFT OUTER JOIN airports AS B
        ON A.DEST = B.IATA
    GROUP BY A.DEST
        , B.Name
        , B.City
        , B.Country
        , B.Latitude
        , B.Longitude
    HAVING COUNT(*) > 1000
        AND B.Country = 'United States'
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 30
''')
ontime_destination.head(5)

In [ ]:
### plot
plot_airports(
    ontime_destination
    , 'Greens'
    , {'CNT': 'ONTIME_CNT', 'AVG': 'AVG_ONTIME'}
    , 'ontime'
)

#### Aerolinea

In [ ]:
%%time
bc.sql('''
    SELECT A.OP_UNIQUE_CARRIER AS CARRIER
        , B.Name AS CARRIER_Name
        , B.Country AS CARRIER_Country
        , AVG(DEP_DELAY) AS AVG_ONTIME
    FROM ontime AS A
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    GROUP BY A.OP_UNIQUE_CARRIER
        , B.Name
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 5
''')

### 5. Qué vuelos que salieron del estado de SEATLE tuvieron retrasos significantes?

In [ ]:
%%time
destinations = bc.sql('''
    SELECT ORIGIN
        , ORIGIN_LAT
        , ORIGIN_LON
        , DEST
        , DEST_LAT
        , DEST_LON
        , COUNT(*) AS DELAY_CNT
        , AVG(Delay) AS AVG_DELAY
    FROM (
        SELECT A.ORIGIN
            , B.Latitude AS ORIGIN_LAT
            , B.Longitude AS ORIGIN_LON
            , A.DEST
            , C.Latitude AS DEST_LAT
            , C.Longitude AS DEST_LON
            , A.DEP_DELAY AS Delay
        FROM flights AS A
        LEFT OUTER JOIN airports AS B
            ON A.ORIGIN = B.IATA
        LEFT OUTER JOIN airports AS C
            ON A.DEST = C.IATA
        WHERE ORIGIN = 'SEA'
            AND C.Country = 'United States'
            AND DEP_DELAY > 60  -- DELAYED BY MORE THAN HOUR
    ) AS A
    GROUP BY ORIGIN
        , ORIGIN_LAT
        , ORIGIN_LON
        , DEST
        , DEST_LAT
        , DEST_LON
    HAVING COUNT(*) > 50
    ORDER BY AVG_DELAY DESC
    LIMIT 30
''')
destinations.head(5)

Graficar los resultados en un mapa

In [ ]:
def add_lines(ax, host):
    lines = host[['ORIGIN_LON', 'ORIGIN_LAT', 'DEST_LON', 'DEST_LAT']].to_dict('list')
    
    sea = (lines['ORIGIN_LON'][0], lines['ORIGIN_LAT'][0])
    for dest in zip(lines['DEST_LON'], lines['DEST_LAT']):
        ax.arrow(*dest
                 , sea[0]-dest[0], sea[1]-dest[1]
                 , head_width=0.0, head_length=0.0
                 , fc='k', ec='k')

host = destinations.to_pandas()
    
### figure
fig = plt.figure(figsize=(12,9))

### plot on map
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([-125, -70, 22, 55], crs=ccrs.PlateCarree()) ## US Continental
ax.stock_img()

### add features
ax.add_feature(cfeature.LAKES)
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.COASTLINE)

### add data
bubbles = ax.scatter(
    host['DEST_LON']
    , host['DEST_LAT']
    , s=host['DELAY_CNT'] * 2, label="Flight Count"
    , c=host['AVG_DELAY']
    , cmap=plt.get_cmap('jet')
    , alpha=0.8
)

add_lines(ax, host)

### add colorbar
delays = host['AVG_DELAY']
tick_values = np.linspace(delays.min(), delays.max(), 11)
cbar = fig.colorbar(bubbles, shrink=0.7)
cbar.ax.set_yticklabels([f'{round(v)} mins' for v in tick_values], fontsize=14)
cbar.set_label(f'Average delays', fontsize=16)
plt.legend(fontsize=16)

### 6. Hay algun avion especifico (por numero de matricula [tail number]) que es menos confiable?

In [ ]:
tail_numbers = bc.sql('''
    SELECT TAIL_NUM
        , B.Name AS AirlineName
        , DEP_DELAY
    FROM flights AS A
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    WHERE TAIL_NUM <> ''
        AND DEP_DELAY > 0
''')

In [ ]:
aircraft_aggs = (
    tail_numbers
    .groupby(by=['TAIL_NUM', 'AirlineName'])
    .agg({'DEP_DELAY': ['count', q25,'median',q75, 'max']})
)

aircraft_aggs.columns = ['_'.join(col) for col in aircraft_aggs.columns]
aircraft_aggs = (
    aircraft_aggs
    .query('DEP_DELAY_count > 100')
    .sort_values(by='DEP_DELAY_median', ascending=False)
)

In [ ]:
aircraft_aggs.head(10)